In [ ]:
import osmnx as ox
from pathlib import Path
import os
import geopandas as gpd

project_dir = Path(os.getcwd()).parent

temp_data_dir = project_dir / "temp_data"
data_dir = project_dir / "data"

cities = ox.features_from_place("Poland", tags={"place": ["city", "town", "village"]})
cities = cities.to_crs(epsg=2180)

cities = cities.loc["node", ["geometry", "name", "place", "population", "name:pl", "wikipedia"]].reset_index(drop=True)
cities["name"] = (
    cities["name"]
        .fillna(cities["name:pl"])
        .fillna(cities["wikipedia"].str.extract(r"^pl:(.*?)\s*\(", expand=False))
)
cities = cities.dropna(subset=["name"])
cities = cities.drop(columns=["name:pl", "wikipedia"])

# cities.to_csv(data_dir / "cities.csv", index=False)

In [ ]:
division = ox.features_from_place(
    "Poland",
    tags={
        "boundary": "administrative",
        "admin_level": ["2", "4", "6"]
    }
)

In [ ]:
# notes FOR FUTURE:

division = division[division.geometry.geom_type.isin(["Polygon", "MultiPolygon"])]
division = division.to_crs(epsg=2180)

poland       = division.loc[division["admin_level"] == "2", ["name", "geometry"]]
voivodeships = division.loc[division["admin_level"] == "4", ["name", "geometry"]]
counties     = division.loc[division["admin_level"] == "6", ["name", "geometry"]]

cities = gpd.sjoin(
    cities,
    voivodeships.rename(columns={"name": "voivodeship"}),
    how="left",
    predicate="within"
)

cities = gpd.sjoin(
    cities,
    counties.rename(columns={"name": "county"}),
    how="left",
    predicate="within"
)

cities = cities.drop(columns=[c for c in cities.columns if c.startswith("index_")])